In [ ]:
# Notebook che permette di estrarre un grafo stradale contenente le vie di una città da mappare al fine di identificare il fenomeno della sosta selvaggia.

# Il notebook prende in input:
# - il grafo stradale di OSM di un area geografica (e.g., tutto il nord ovest dell'italia)
# - un file contentente i confini poligonali di un'area circoscritta (e.g., la città di milano)

# Il notebook restituisce in output:
# - il grafo stradale relativo alla sola area circoscritta (e.g., la città di milano) preprocessato (contenente solo le strade da mappare poiché possono ospitare la sosta di auto)


In [ ]:
import pandas as pd
import geopandas as gpd


# input

In [ ]:

# INPUT:
# a) grafo stradale di OSM scaricato dal sito https://download.geofabrik.de/
grafo_stradale_OSM_global  = "data\\grafo_OSM_italy_nord_ovest\\gis_osm_roads_free_1.shp"

# b) road_type_to_keep_list --> lista che contiene solo le strade con categoria (fclass) a cui sono interessato (quelle carrozzabili)
# questa lista è dedotta a partire da https://wiki.openstreetmap.org/wiki/IT:Key:highway
road_type_to_keep_list = [
 # 'emergency_bay', # togliere
 'living_street',
#  'motorway', # togliere tangenziali
#  'motorway_link', # togliere tangenziali
 'primary',
 'primary_link',
 'residential',
 'secondary',
 'secondary_link',
 'tertiary',
 'tertiary_link',
#  'trunk', # togliere cavalcavia
#  'trunk_link', # togliere cavalcavia
 'unclassified'
]

# c) road_name_to_exclude  --> lista che contiene i nomi delle vie che voglio escludere poichè da non mappare
road_name_list_to_exclude = [
'Via Monte Spluga',
'Via Fratelli Beltrami',
'Via Giovanni Battista Grassi',
'Via Cristina Belgioioso',
'Via Monte Santo',
"Via dell'Innovazione",
'Collegamento Via Eritrea - Via Stephenson',
'Collegamento Via Barzaghi - Via Stephenson',
'Rotatoria Barzaghi',
'Rotatoria Merlata',
'Via Cusago',
'Via Sandro Pertini',
'Via Cascina Nuova',
'Via Vaiano Valle',
'Cavalcavia Buccari',
'Cavalcavia Pontinia',
'Cavalcavia Adriano Bacula',
'Cavalcavia Buccari',
'Cavalcavia Don Lorenzo Milani',
'Cavalcavia Giovanni Schiavoni',
'Cavalcavia Giuseppe Luraghi',
'Cavalcavia Pontinia',
'Via Stigliano',
'Via Settimo Milanese',
'Via Riccardo Lombardi',
'Via dei Giovi',
'Via Virgilio Ferrari',
'Viale Enrico Fermi',
'Via Cassanese',
'Viale Turchia',
'Via Costante Giardegno',
'Via dei Giovi',
'Via Gattinara',
'Strada Comunale Chiaravalle',
'Viale Enrico Forlanini',
'Via Assiano',
'Via Guascona',
'Via Martirano',
'Strada per Baggio',
'Via Valpolicella',
'Via Bardolino',
'Via San Marchetto',
'Via Marignano',
'Via Milano',
'Sottopasso Patrocio',
'Sottopasso Patroclo', 
'Via privata Pienza',
'Via Papa Giovanni Ventitreesimo',
'Via Camillo Sbarbaro',
'Via F.lli Rizzardi - Milano',
'Via Costante Girardengo'
]
# d) osmid_list_to_exclude  --> lista che contiene gli osm_id - link stradali - che voglio escludere poichè da non mappare
osmid_list_to_exclude = ['48957470', '48957819','266577721','24725610','271897104','141209315','266587556','1207691444','444928122','1207689259','266587555','81201883','1207689260','1207689261','81201869','23275705','926161090','1044592784','282049361','282049359','305112135','305697007','25736767','1218322249','50097374','273918540','244682940','25121323']

# e) geometria della città (poligono) che voglio estrarre rispetto al grafo stradale di OSM di input
poligono_confini_comunali = "data\poligoni_milano\A090101_ComuneMilano.shp"

# f) nome finale grafo comune in output
final_name_grafo_comune_filtered = 'grafo_stradale_OSM_comune_filtered'


# main

In [ ]:
gdf_grafo_global = gpd.read_file(grafo_stradale_OSM_global)
gdf_grafo_global.head(2)

In [ ]:
gdf_grafo_global.crs

In [ ]:
gdf_grafo_global_filtered = gdf_grafo_global[gdf_grafo_global['fclass'].isin(road_type_to_keep_list)]
gdf_grafo_global_filtered= gdf_grafo_global_filtered[~gdf_grafo_global_filtered['osm_id'].isin(osmid_list_to_exclude)]
gdf_grafo_global_filtered= gdf_grafo_global_filtered[~gdf_grafo_global_filtered['name'].isin(road_name_list_to_exclude)]

In [ ]:
gdf_poligono_comune= gpd.read_file(poligono_confini_comunali)
gdf_poligono_comune.head(2)


In [ ]:
gdf_poligono_comune.crs

In [ ]:
# proietto su piano riferimento italia roma 3003 prima di spatial join
gdf_grafo_global_filtered = gdf_grafo_global_filtered.to_crs(epsg=3003) # spaltial projection on Monte Mario / Italy zone 1
gdf_poligono_comune = gdf_poligono_comune.to_crs(epsg=3003) # spaltial projection on Monte Mario / Italy zone 1

# spatial join
column_list = list(gdf_grafo_global_filtered.columns)
column_list.remove('geometry')

join_intersect =  gpd.sjoin(gdf_poligono_comune, gdf_grafo_global_filtered,predicate='intersects', how = 'left').reset_index(drop=True)# linee che intersecano (sia between che interno) i poligono
gpd_road_comune_final = gpd.GeoDataFrame(join_intersect[column_list].merge(gdf_grafo_global_filtered[['osm_id','geometry']],how='left'))


In [ ]:
gpd_road_comune_final.head(2)

In [ ]:
gpd_road_comune_final.plot()

In [ ]:
gpd_road_comune_final.to_file(f"data\\{final_name_grafo_comune_filtered}.shp") 